In [1]:
import numpy as np
import cv2

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
# config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.8
session = InteractiveSession(config=config)

from srgan2 import SRGAN
from generator import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# !TF_ENABLE_GPU_GARBAGE_COLLECTION=false

In [3]:
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [4]:
r = 2
input_dir = f'data/train/{r}/low/'
tgt_dir = f'data/train/{r}/high/'

input_dir_val = f'data/val/{r}/low/'
tgt_dir_val = f'data/val/{r}/high/'


test_file = 'UBMk30rjy0o_17675_42.jpg'
test_low_path =  input_dir_val + test_file
test_high_path = tgt_dir_val + test_file
test_out_path = 'data/out/' + test_file
test_test_path = 'data/test/' + test_file

vgg_path = 'models/vgg16_notop.hdf5'

input_shape = (67, 120, 3)
tgt_shape = (134, 240, 3)

batch_size = 16

In [5]:
srgan = SRGAN(vgg_path, batch_size, input_shape, tgt_shape)
gen = ImageDataGenerator(input_dir, tgt_dir, batch_size)
gen_valid = ImageDataGenerator(input_dir_val, tgt_dir_val, batch_size)

/home/keras/srgan2.py:102: UserWarning: Update your `Network` call to the Keras 2 API: `Network(name="generator", inputs=Tensor("in..., outputs=Tensor("ou...)`
  self.shared_generator = Network(input=layers[0], output=layers[-1], name='generator')
/home/keras/srgan2.py:131: UserWarning: Update your `Network` call to the Keras 2 API: `Network(name="discriminator", inputs=Tensor("in..., outputs=Tensor("de...)`
  self.shared_discriminator = Network(input=layers[0], output=layers[-1], name='discriminator')
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
for e in range(2001):
    X_low, X_high = gen.flow_from_directory().__next__()
    srgan.train(X_low, X_high)
    if e % 100 == 0:
        print('epoch', e)
        srgan.valid(X_low, X_high)
        
        test_img_low = ((cv2.imread(test_low_path) - 127.5) / 127.5)[np.newaxis, :, :, :]
        test_img_high = ((cv2.imread(test_high_path)- 127.5) / 127.5)[np.newaxis, :, :, :]

        test_img_sr = srgan.generator.predict(test_img_low)
        test_img_sr_int = ((test_img_sr * 127.5) + 127.5).astype('u1')[0]
        test_out_path = 'data/out/'  + test_file.split('.')[0] + f'_test_{e}.jpg'
        cv2.imwrite(test_out_path, test_img_sr_int)

epoch 0
d_loss_real:1.348144769668579, d_loss_fake:0.0859028548002243, vgg_loss:27.327592849731445
epoch 100
d_loss_real:0.713306188583374, d_loss_fake:0.6655640602111816, vgg_loss:6.495380878448486
epoch 200
d_loss_real:0.6893531084060669, d_loss_fake:0.6879932880401611, vgg_loss:7.170319557189941
epoch 300
d_loss_real:0.6710020303726196, d_loss_fake:0.7160060405731201, vgg_loss:4.387297630310059
epoch 400
d_loss_real:0.6943860054016113, d_loss_fake:0.6786378026008606, vgg_loss:3.887230634689331
epoch 500
d_loss_real:0.7051687836647034, d_loss_fake:0.672368586063385, vgg_loss:5.469685077667236
epoch 600
d_loss_real:0.6632356643676758, d_loss_fake:0.6883395910263062, vgg_loss:3.349947690963745
epoch 700
d_loss_real:0.7061704397201538, d_loss_fake:0.683529794216156, vgg_loss:4.129391670227051
epoch 800
d_loss_real:0.6782545447349548, d_loss_fake:0.6791698932647705, vgg_loss:3.3970985412597656
epoch 900
d_loss_real:0.6824143528938293, d_loss_fake:0.685114860534668, vgg_loss:3.66067218780

In [7]:
srgan.generator.save('models/20200515_generator.h5', include_optimizer=False)